In [ ]:
# Download files from https://towardsdatascience.com/springer-has-released-65-machine-learning-and-data-books-for-free-961f8181f189

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [2]:
url = "https://towardsdatascience.com/springer-has-released-65-machine-learning-and-data-books-for-free-961f8181f189"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
links = soup.findAll('table',{'class':'cq gv ie if ig ih'})

In [28]:
links = []
for link in soup.findAll('a'):
    l = link.get('href')
    if l.startswith('http://link.springer.com'):
        links.append(l)

In [ ]:
for link in links:
    

In [33]:
import requests
url=links[0] + ".pdf"
r = requests.get(url, stream=True)

with open('../Books/myfile.pdf', 'wb') as f:
    f.write(r.content)


In [32]:
url

'http://link.springer.com/openurl?genre=book&isbn=978-0-387-84858-7'

In [34]:
import os
import requests
import shutil


def create_relative_path_if_not_exist(relative_path):
    path = os.path.join(os.getcwd(), relative_path)
    if not os.path.exists(path):
        os.makedirs(path)
    return path


def download_book(url, bookpath):
    if not os.path.exists(bookpath):
        with requests.get(url, stream = True) as req:
            path = create_relative_path_if_not_exist('tmp')
            tmp_file = os.path.join(path, '_-_temp_file_-_.bak')
            with open(tmp_file, 'wb') as out_file:
                shutil.copyfileobj(req.raw, out_file)
                out_file.close()
            shutil.move(tmp_file, bookpath)


replacements = {'/':'-', '\\':'-', ':':'-', '*':'', '>':'', '<':'', '?':'', \
                '|':'', '"':''}


def compose_bookname(title, author, edition, isbn):
    bookname = title + ' - ' + author + ', ' + edition + ' - ' + isbn
    if(len(bookname) > 145):
        bookname = title + ' - ' + author.split(',')[0] + ' et al., ' + \
                    edition + ' - ' + isbn
    if(len(bookname) > 145):
        bookname = title + ' - ' + author.split(',')[0] + ' et al. - ' + isbn
    if(len(bookname) > 145):
        bookname = title + ' - ' + isbn
    if(len(bookname) > 145):
        bookname = title[:130] + ' - ' +isbn
    bookname = bookname.encode('ascii', 'ignore').decode('ascii')
    return "".join([replacements.get(c, c) for c in bookname])

In [41]:
import os
import requests
import pandas as pd
import time
from tqdm import tqdm
# from helper import *

folder = "../Books/" # create_relative_path_if_not_exist('downloads')

table_url = 'https://resource-cms.springernature.com/springer-cms/rest/v1/content/17858272/data/v4'
table = 'table_' + table_url.split('/')[-1] + '.xlsx'
table_path = os.path.join(folder, table)
if not os.path.exists(table_path):
    books = pd.read_excel(table_url)
    # Save table
    books.to_excel(table_path)
else:
    books = pd.read_excel(table_path, index_col=0, header=0)


for url, title, author, edition, isbn, category in tqdm(books[['OpenURL', 'Book Title', 'Author', 'Edition', 'Electronic ISBN', 'English Package Name']].values):
    new_folder = create_relative_path_if_not_exist(os.path.join(folder, category))

    bookname = compose_bookname(title, author, edition, isbn)
    output_file = os.path.join(new_folder, bookname + '.pdf')

    # If book already downloaded, skip it
    if os.path.exists(output_file):
        continue

    try:
        r = requests.get(url)
        new_url = r.url.replace('%2F','/').replace('/book/','/content/pdf/') + '.pdf'
        download_book(new_url, output_file)

        # Download EPUB version too if exists
        new_url = r.url.replace('%2F','/').replace('/book/','/download/epub/') + '.epub'
        output_file = os.path.join(new_folder, bookname + '.epub')
        request = requests.get(new_url, stream = True)
        if request.status_code == 200:
            download_book(new_url, output_file)
    except:
        print('\nProblem downloading: ' + title)
        time.sleep(30)
        continue

print('\nFinish downloading.')

100%|██████████| 407/407 [54:21<00:00,  8.01s/it]   


Finish downloading.


In [43]:
import glob

In [51]:
glob.glob("../Books/*/*Calculus*")

['../Books/Mathematics and Statistics/Calculus With Applications - Peter D. Lax, Maria Shea Terrell, 2nd ed. 2014 - 978-1-4614-7946-8.pdf',
 '../Books/Mathematics and Statistics/Calculus With Applications - Peter D. Lax, Maria Shea Terrell, 2nd ed. 2014 - 978-1-4614-7946-8.epub',
 '../Books/Mathematics and Statistics/Multivariate Calculus and Geometry - Sen Dineen, 3rd ed. 2014 - 978-1-4471-6419-7.epub',
 '../Books/Mathematics and Statistics/Brownian Motion, Martingales, and Stochastic Calculus  - Jean-Franois Le Gall, 1st ed. 2016 - 978-3-319-31089-3.epub',
 '../Books/Mathematics and Statistics/Brownian Motion, Martingales, and Stochastic Calculus  - Jean-Franois Le Gall, 1st ed. 2016 - 978-3-319-31089-3.pdf',
 '../Books/Mathematics and Statistics/Multivariate Calculus and Geometry - Sen Dineen, 3rd ed. 2014 - 978-1-4471-6419-7.pdf',
 '../Books/Economics and Finance/Stochastic Processes and Calculus - Uwe Hassler, 1st ed. 2016 - 978-3-319-23428-1.pdf',
 '../Books/Economics and Finance